In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import cv2
from zipfile import ZipFile
import os
import random
from sklearn.model_selection import train_test_split
from data_augment import augment_data
from collections import Counter

In [ ]:
image_size = 128
num_channels = 3
num_classes = 120
num_input = image_size*image_size
learning_rate = 0.00001

batch_size = 256
test_size = 250
num_iter = 30
num_epoch = 100
#dropout = 1

In [ ]:
dir_name = 'C:/Users/User/Downloads/Capstone Project/'
filenames1 = ZipFile('C:/Users/User/Downloads/Capstone Project/train.zip','r') 
train = filenames1.namelist()
train = [os.path.join(dir_name,i) for i in train]
train = train[1:]

labels = pd.read_csv("C:/Users/User/Downloads/Capstone Project/labels.csv")

label = pd.get_dummies(list(labels['breed']))

X_train, X_test, y_train, y_test = train_test_split(train, label, test_size=0.10, random_state=42)

In [ ]:
def generate_batch(train,label,step):
    filename = train[step*batch_size :(step+1)*batch_size]
    batch_y = label[step*batch_size :(step+1)*batch_size]
    batch_x = [cv2.imread(i)for i in filename]
    batch_x = [cv2.resize(i,(image_size,image_size))for i in batch_x]
    batch_x = np.array(batch_x)
    batch_x = np.reshape(batch_x,(batch_size,image_size,image_size,3))
    
    return batch_x, batch_y

def generate_test(test,label):
    
    filename = test[0 : test_size]
    batch_y = label[0 : test_size]
    batch_x = [cv2.imread(i)for i in filename]
    batch_x = [cv2.resize(i,(image_size,image_size))for i in batch_x]
    batch_x = np.array(batch_x)
    batch_x = np.reshape(batch_x,(len(filename),image_size,image_size,3))
    
    return batch_x,batch_y

test_x,test_y = generate_test(X_test, y_test)

In [ ]:
# tf Graph input
X = tf.placeholder(tf.float32, [None, image_size,image_size,num_channels])
Y = tf.placeholder(tf.float32, [None, num_classes])
# dropout (keep probability)
#keep_prob = tf.placeholder(tf.float32)

In [ ]:
#def conv2d(x, W, b, strides=1):
#    # Conv2D wrapper, with bias and relu activation
#    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
#    x = tf.nn.bias_add(x, b)
#    return tf.nn.relu(x)

def conv2d(x, shape, num,strides=1):
   w = tf.get_variable('w' + str(num), shape = shape, initializer = tf.contrib.layers.xavier_initializer(seed=0))
   b = tf.zeros(shape[-1], name='b' + str(num))
   conv = tf.nn.conv2d(x, w, [1,strides,strides,1], padding="SAME")
   conv = tf.nn.bias_add(conv, b)
   return(tf.nn.relu(conv))

In [ ]:
conv1 = conv2d(X, [3,3,3,16], 1)
conv2 = conv2d(conv1, [3,3,16,16], 2)
maxp1 = tf.nn.max_pool(conv2, [1,2,2,1], [1,2,2,1], padding='SAME')

#bn1 = tf.layers.batch_normalization(maxp1)

conv3 = conv2d(maxp1, [5,5,16,32], 3)
conv4 = conv2d(conv3, [5,5,32,32], 4)
maxp2 = tf.nn.max_pool(conv4, [1,2,2,1], [1,2,2,1], padding='SAME')

fc = tf.contrib.layers.flatten(maxp2)
#kp    = tf.placeholder(tf.float32)
#fc    = tf.nn.dropout(fc, keep_prob)

fc1 = tf.contrib.layers.fully_connected(fc, 4096,activation_fn=None)

bn2 = tf.layers.batch_normalization(fc1)
bn2 = tf.nn.relu(bn2)

fc2 = tf.contrib.layers.fully_connected(bn2, 2048)
#fc2   = tf.nn.dropout(fc2, keep_prob)

fc3 = tf.contrib.layers.fully_connected(fc, 120, activation_fn=None)

In [ ]:
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = fc3, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_op)

prediction = tf.nn.softmax(fc3)
# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

filename = []

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [ ]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    
    
    
    for epoch in range(num_epoch):
        
        label1 = y_train.values.tolist()
        c = list(zip(train, label1))
        
        random.shuffle(c)

        train, label1 = zip(*c)
        train = list(train)
        label1 = np.array(label1)
        label2 = pd.DataFrame(np.array(label1))
        label2.columns = list(label.columns.values)
        
        for step in range(num_iter):
            
            batch_x,batch_y = generate_batch(X_train, label2,step)
            batch_x = augment_data(batch_x)
            batch_x = batch_x/255
            
            # Run optimization op (backprop)
            _,loss,acc = sess.run([optimizer,loss_op, accuracy], feed_dict={X: batch_x,
                                                                     Y: batch_y})
                                                                     
            
            if(step%10 == 0):
                print("Epoch " + str(epoch) + " Iteration " + str(step) + " Train Loss= " + \
                      str(loss) + " Train Accuracy= " + \
                      str(acc))
                test_acc,test_pred=sess.run([accuracy,tf.argmax(prediction,1)], feed_dict={X: test_x/255,Y: test_y})
                print(" Testing Accuracy:" + str(test_acc) + " Testing Pred:" + str(Counter(test_pred).most_common(3)))                    
                                      
                                      
            

    print("Optimization Finished!")